### Paths & Environment

In [1]:
import os
DATA_DIR   = "/apps/data/share/LaSOT14_Dataset"               
TRAIN_JSON = "/apps/data/share/LaSOT14_Dataset/annotations/train/train.json"
TEST_JSON  = "/apps/data/share/LaSOT14_Dataset/annotations/test/test.json"
VAL_JSON  = "/apps/data/share/LaSOT14_Dataset/annotations/val/val.json"
LOCAL_DETR = "/apps/data/share/models/deformable_detr_coco"
REPO_OSTRACK     = "/apps/data/share/repos/OSTrack"
REPO_TRACKFORMER = "/apps/data/share/repos/TrackFormer"
REPO_MOTRV2      = "/apps/data/share/repos/MOTRv2"
os.environ["LOCAL_DEFORMABLE_DETR_PATH"] = LOCAL_DETR

print("Paths set. Edit above if needed.") 

Paths set. Edit above if needed.


### Dependencies checking

In [2]:
import torch, torchvision, transformers, cv2
print("torch:", torch.__version__)          
print("torchvision:", torchvision.__version__)
print("transformers:", transformers.__version__)
print("opencv:", cv2.__version__)

torch: 2.9.0+cu128
torchvision: 0.24.0+cu128
transformers: 4.57.1
opencv: 4.12.0


In [10]:
%%bash
# Reset allocator envs (remove old lowercase 'true') 
unset PYTORCH_CUDA_ALLOC_CONF
unset PYTORCH_ALLOC_CONF

# Good allocator config (booleans must be 'True'/'False')  
export PYTORCH_ALLOC_CONF='expandable_segments:True,max_split_size_mb:128,garbage_collection_threshold:0.8'

# Optional NCCL safety  
export TORCH_NCCL_BLOCKING_WAIT=1
export TORCH_NCCL_ASYNC_ERROR_HANDLING=1
export NCCL_IB_DISABLE=1

# Show what will be inherited  
echo "PYTORCH_ALLOC_CONF=$PYTORCH_ALLOC_CONF"

# Kill your own leftovers only  
pkill -f "torchrun .*Project2_SOFT/train.py"  || true
pkill -f "python .*Project2_SOFT/train.py"    || true


PYTORCH_ALLOC_CONF=expandable_segments:True,max_split_size_mb:128,garbage_collection_threshold:0.8


### Train our model 

In [1]:
set -e

CUDA_VISIBLE_DEVICES=2,3,4,5,6,7 \
OMP_NUM_THREADS=1 \
TORCH_NCCL_BLOCKING_WAIT=1 \
TORCH_NCCL_ASYNC_ERROR_HANDLING=1 \
NCCL_IB_DISABLE=1 \
MASTER_ADDR=127.0.0.1 \
MASTER_PORT=29509 \
PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True,max_split_size_mb:128,garbage_collection_threshold:0.8' \
torchrun --standalone --nproc_per_node=6 train_improved.py \
  --data_dir   /apps/data/share/LaSOT14_Dataset \
  --train_json /apps/data/share/LaSOT14_Dataset/annotations/train/train.json \
  --val_json   /apps/data/share/LaSOT14_Dataset/annotations/val/val.json \
  --output_dir ../ckpts/run_improved_v1 \
  --epochs 100 \
  --batch_size 1 \
  --segment_len 24 \
  --tail_len 24 \
  --variant temporal_primed \
  --k_burnin 3 \
  --prime_p_start 1.0 \
  --prime_p_end 0.7 \
  --prime_decay_epochs 60 \
  --lr_heads 2e-4 \
  --lr_backbone 2e-5 \
  --weight_decay 1e-4 \
  --short_side 768 \
  --max_side 1024 \
  --num_queries 300 \
  --early_stop_patience 15 \
  --plateau_patience 5 \
  --plateau_factor 0.5 \
  --clip_grad_norm 0.5 \
  --gt_fmt auto \
  --diag_norm 0.005 \
  --p20 20 \
  --find_unused \
  --augment

### Ablations 

In [2]:
# 1) Frame-only (no temporal, no priming, no anchor) 
!torchrun --standalone --nproc_per_node=8 ../train.py \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" \
  --output_dir ../ckpts/frame_only --epochs 10 --batch_size 8 \
  --segment_len 30 --future_len 10 --variant frame_only

# 2) + Temporal (no priming, no anchor) 
!torchrun --standalone --nproc_per_node=8 ../train.py \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" \
  --output_dir ../ckpts/temporal --epochs 10 --batch_size 8 \
  --segment_len 30 --future_len 10 --variant temporal

# 3) + GT-Primed (still no anchor) 
!torchrun --standalone --nproc_per_node=8 ../train.py \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" \
  --output_dir ../ckpts/temporal_primed --epochs 10 --batch_size 8 \
  --segment_len 30 --future_len 10 --variant temporal_primed

# 4) + Burn-in Anchor Loss 
!torchrun --standalone --nproc_per_node=8 ../train.py \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" \
  --output_dir ../ckpts/temporal_primed_anchor --epochs 10 --batch_size 8 \
  --segment_len 30 --future_len 10 --variant temporal_primed_anchor --w_anchor 10

### Inference 

In [ ]:
# Note: during inference, we disable priming; choose "temporal" or "frame_only" path 
!torchrun --standalone --nproc_per_node=8 ../test.py \
  --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" \
  --checkpoint ../ckpts/full/checkpoint_final.pth \
  --output_dir ../results/full --variant temporal

# Frame-only
!torchrun --standalone --nproc_per_node=8 ../test.py \
  --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" \
  --checkpoint ../ckpts/frame_only/checkpoint_final.pth \
  --output_dir ../results/frame_only --variant frame_only

# Temporal
!torchrun --standalone --nproc_per_node=8 ../test.py \
  --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" \
  --checkpoint ../ckpts/temporal/checkpoint_final.pth \
  --output_dir ../results/temporal --variant temporal

# Temporal + Primed
!torchrun --standalone --nproc_per_node=8 ../test.py \
  --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" \
  --checkpoint ../ckpts/temporal_primed/checkpoint_final.pth \
  --output_dir ../results/temporal_primed --variant temporal

# Temporal + Primed + Anchor
!torchrun --standalone --nproc_per_node=8 ../test.py \
  --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" \
  --checkpoint ../ckpts/temporal_primed_anchor/checkpoint_final.pth \
  --output_dir ../results/temporal_primed_anchor --variant temporal

### Evaluation (AUC / Precision / ADE / FDE)

In [ ]:
# Evaluate five result folders; print metrics for table copy 
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../results/full
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../results/frame_only
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../results/temporal
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../results/temporal_primed
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../results/temporal_primed_anchor

### Visualize one sequence

In [ ]:
# Pick one test sequence name from your LaSOT split 
SEQ = "airplane-1"  # change accordingly 

# Draw Pred (red) and GT (green, if available) and export MP4 
!python ../visualize_results.py --sequence "{SEQ}" --data_dir "{DATA_DIR}" \
  --result_dir ../results/full --output_video ../vis_{SEQ}.mp4 --show_gt

### External baselines (OSTrack / TrackFormer / MOTRv2)

In [ ]:
# Ensure repos are cloned and deps installed per each README 
# Each run: train -> infer -> export <seq>.txt into work_dir/results 
# OSTrack
!python ../baselines_runner.py --model ostrack --repo_root "{REPO_OSTRACK}" \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" --test_json "{TEST_JSON}" \
  --work_dir ../work/ostrack --gpus 8
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../work/ostrack/results

# TrackFormer
!python ../baselines_runner.py --model trackformer --repo_root "{REPO_TRACKFORMER}" \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" --test_json "{TEST_JSON}" \
  --work_dir ../work/trackformer --gpus 8
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../work/trackformer/results

# MOTRv2
!python ../baselines_runner.py --model motrv2 --repo_root "{REPO_MOTRV2}" \
  --data_dir "{DATA_DIR}" --train_json "{TRAIN_JSON}" --test_json "{TEST_JSON}" \
  --work_dir ../work/motrv2 --gpus 8
!python ../eval_metrics.py --data_dir "{DATA_DIR}" --test_json "{TEST_JSON}" --result_dir ../work/motrv2/results